1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./tax.docx")
documents = loader.load_and_split(text_splitter=text_splitter)

In [11]:
len(documents)

220

In [1]:
%pip install langchain_upstage

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large')

In [9]:
%pip install langchain-chroma

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 9.3 MB/s eta 0:00:0031m9.4 MB/s eta 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 11.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.2 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 12.8 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 9.7 MB/s eta 0:00:00
  Created wheel for pypika: filename=PyPika-0.48.9-py2.py3-none-any.whl size=53726 sha256=520d5f63c5245a601b17a5c67911984bc60cf93e0689e682de265a89d60e7bfe
  Stored in directory: /Users/choichiwon/Library/Caches/pip/wheels/e1/26/51/d0bffb

In [5]:
from langchain_chroma import Chroma

database = Chroma.from_documents(documents=documents, embedding=embedding, persist_directory="./chroma", collection_name='chroma-tax')
# database = Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding_function=embedding)

In [6]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

retrieved_docs = database.similarity_search(query, k=3)

In [7]:
retrieved_docs

[Document(metadata={'source': './tax.docx'}, page_content='1. 총급여액이 3천 300만원 이하인 경우: 74만원\n\n2. 총급여액이 3천 300만원 초과 7천만원 이하인 경우: 74만원 - [(총급여액 - 3천 300만원) × 8/1000]. 다만, 위 금액이 66만원보다 적은 경우에는 66만원으로 한다.\n\n3. 총급여액이 7천만원 초과 1억2천만원 이하인 경우: 66만원 - [(총급여액 - 7천만원) × 1/2]. 다만, 위 금액이 50만원보다 적은 경우에는 50만원으로 한다.\n\n4. 총급여액이 1억2천만원을 초과하는 경우: 50만원 - [(총급여액 - 1억2천만원) × 1/2]. 다만, 위 금액이 20만원보다 적은 경우에는 20만원으로 한다.\n\n③ 일용근로자의 근로소득에 대해서 제134조제3항에 따른 원천징수를 하는 경우에는 해당 근로소득에 대한 산출세액의 100분의 55에 해당하는 금액을 그 산출세액에서 공제한다.<개정 2014. 1. 1.>\n\n[전문개정 2012. 1. 1.]\n\n\n\n제59조의2(자녀세액공제) ①종합소득이 있는 거주자의 기본공제대상자에 해당하는 자녀(입양자 및 위탁아동을 포함하며, 이하 이 조에서 “공제대상자녀”라 한다) 및 손자녀로서 8세 이상의 사람에 대해서는 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다. <개정 2015. 5. 13., 2017. 12. 19., 2018. 12. 31., 2019. 12. 31., 2022. 12. 31., 2023. 12. 31.>\n\n1. 1명인 경우: 연 15만원\n\n2. 2명인 경우: 연 35만원\n\n3. 3명 이상인 경우: 연 35만원과 2명을 초과하는 1명당 연 30만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016

In [8]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

In [9]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [10]:
ai_response = llm.invoke(prompt)

In [11]:
ai_response.content

'연봉 5천만원인 직장인의 소득세는 약 1,190만원입니다.'

'연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 기본세율을 적용해야 합니다. 기본세율은 과세표준에 따라 다르며, 다음과 같습니다:\n\n1. 과세표준 1,200만원 이하: 6%\n2. 과세표준 1,200만원 초과 4,600만원 이하: 15%\n3. 과세표준 4,600만원 초과 8,800만원 이하: 24%\n4. 과세표준 8,800만원 초과 1억5천만원 이하: 35%\n5. 과세표준 1억5천만원 초과 3억원 이하: 38%\n6. 과세표준 3억원 초과 5억원 이하: 40%\n7. 과세표준 5억원 초과: 42%\n\n연봉 5천만원의 경우, 과세표준은 다음과 같이 계산됩니다:\n\n1. 연봉 5천만원에서 근로소득공제와 각종 공제를 차감한 금액이 과세표준이 됩니다.\n2. 근로소득공제는 다음과 같습니다:\n   - 연봉 5천만원의 경우: 3,600만원 + (연봉 - 4,500만원) * 5% = 3,600만원 + (5,000만원 - 4,500만원) * 5% = 3,600만원 + 25만원 = 3,625만원\n\n따라서, 과세표준은 5천만원 - 3,625만원 = 1,375만원입니다.\n\n이제, 기본세율표를 적용하여 소득세를 계산합니다:\n\n- 과세표준 1,200만원 이하 부분: 1,200만원 * 6% = 72만원\n- 과세표준 1,200만원 초과 1,375만원 이하 부분: (1,375만원 - 1,200만원) * 15% = 175만원 * 15% = 26.25만원\n\n따라서, 총 소득세는 72만원 + 26.25만원 = 98.25만원입니다.\n\n연봉 5천만원인 직장인의 소득세는 대략 98.25만원이 됩니다. 다만, 이 계산은 근로소득공제와 기본세율만을 고려한 것이며, 실제 소득세는 각종 세액공제, 보험료 공제 등 다른 공제를 고려해야 합니다. 정확한 금액은 국세청 홈택스 등의 소득세 계산기를 이용해 보시는 것이 좋습니다.'

In [12]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/Users/choichiwon/.pyenv/versions/helloworld/lib/python3.10/site-packages/langsmith/client.py:322: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/choichiwon/.pyenv/versions/helloworld/lib/python3.10/site-packages/langsmith/client.py:5301: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [16]:
ai_message = qa_chain({"query": query})

/var/folders/54/2vnvnn154r5_5fwkj7dvqtn00000gn/T/ipykernel_26684/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  ai_message = qa_chain({"query": query})


In [17]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 소득세는 157만원입니다.'}